# GroundedDINO Annotation Utilities Demo

This notebook demonstrates how to use the `grounded_sam_utils.py` module to:
1. Load the GroundingDINO model.
2. Create and manage a structured JSON file for annotations.
3. Run inference on images with different prompts.
4. Save, retrieve, and visualize annotations for the same image from multiple prompts.

In [ ]:
# Cell 1: Setup and Imports
import os
import sys
from pathlib import Path
import torch

# Add project root to path to allow importing utils
SANDBOX_ROOT = Path("/net/trapnell/vol1/home/mdcolon/proj/morphseq/segmentation_sandbox")
if str(SANDBOX_ROOT) not in sys.path:
    sys.path.append(str(SANDBOX_ROOT))a

# Import the new utilities
from scripts.utils.grounded_sam_utils import (
    load_config,
    load_groundingdino_model,
    GroundedDinoAnnotations,
    run_inference,
    visualize_detections,
    inference_and_visualize
)
from scripts.utils.experiment_metadata_utils import load_experiment_metadata, get_image_id_paths

print("✅ Utilities imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✅ Utilities imported successfully!
PyTorch version: 2.5.1
CUDA available: False


# Cell 2: Load Models and Data

# Load pipeline configuration
config_path = SANDBOX_ROOT / "configs" / "pipeline_config.yaml"
config = load_config(config_path)
print("✅ Pipeline config loaded.")

# Load experiment metadata
experiment_metadata_path = SANDBOX_ROOT / "data" / "raw_data_organized" / "experiment_metadata.json"
experiment_metadata = load_experiment_metadata(experiment_metadata_path)
print("✅ Experiment metadata loaded.")

# Load GroundingDINO model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = load_groundingdino_model(config, device=device)

# Initialize the annotation manager
annotations_path = SANDBOX_ROOT / "data" / "intermediate" / "demo_annotations.json"
if annotations_path.exists():
    annotations_path.unlink() # Start fresh for the demo
    print(f"Removed existing demo annotations file.")
    
annotations_manager = GroundedDinoAnnotations(annotations_path, experiment_metadata)
print("✅ Annotation manager initialized.")

### Step 1: Run Inference and Save First Annotation

# Cell 3: Run Inference on a Sample Image

# Select a sample image
sample_image_id = experiment_metadata["image_ids"][100] # Pick an arbitrary image
image_path = get_image_id_paths(sample_image_id, experiment_metadata)
print(f"Selected image: {sample_image_id}")
print(f"Image path: {image_path}")

# Define the first prompt
prompt_embryo = "zebrafish embryo"
detection_params = {"box_threshold": 0.4, "text_threshold": 0.25}

# Run inference
boxes, logits, phrases, image_source = run_inference(
    model,
    image_path,
    prompt_embryo,
    box_threshold=detection_params["box_threshold"],
    text_threshold=detection_params["text_threshold"]
)

print(f"Found {len(boxes)} detections for prompt: '{prompt_embryo}'")

# Cell 4: Add Annotation to Storage and Save

# Define the model config for this annotation
model_config = {
    "model": "GroundingDINO_SwinT_OGC",
    "box_threshold": detection_params["box_threshold"],
    "text_threshold": detection_params["text_threshold"]
}

# Add the annotation
annotations_manager.add_annotation(
    image_id=sample_image_id,
    prompt=prompt_embryo,
    model_config=model_config,
    boxes=boxes.cpu().numpy(),
    logits=logits.cpu().numpy(),
    phrases=phrases
)

# Save the annotations file
annotations_manager.save()

### Step 2: Run Inference with a Different Prompt on the Same Image

# Cell 5: Run Inference with a Second Prompt

prompt_yolk = "yolk"

boxes_yolk, logits_yolk, phrases_yolk, _ = run_inference(
    model,
    image_path,
    prompt_yolk,
    box_threshold=0.3,
    text_threshold=0.25
)

print(f"Found {len(boxes_yolk)} detections for prompt: '{prompt_yolk}'")

# Add this second annotation to the same image
yolk_model_config = {"model": "GroundingDINO_SwinT_OGC", "box_threshold": 0.3, "text_threshold": 0.25}

annotations_manager.add_annotation(
    image_id=sample_image_id,
    prompt=prompt_yolk,
    model_config=yolk_model_config,
    boxes=boxes_yolk.cpu().numpy(),
    logits=logits_yolk.cpu().numpy(),
    phrases=phrases_yolk
)

# Save again
annotations_manager.save()

### Step 3: Retrieve and Visualize All Annotations for the Image

In [ ]:
# Cell 6: Retrieve and Visualize Stored Annotations

# Retrieve all annotations for the image
all_image_annotations = annotations_manager.get_annotations_for_image(sample_image_id)
print(f"Found {len(all_image_annotations)} stored annotation sets for image {sample_image_id}.")

# Loop through and visualize each one
for i, annotation_set in enumerate(all_image_annotations):
    prompt = annotation_set['prompt']
    num_detections = annotation_set['num_detections']
    print(f"\n--- Visualizing Annotation Set {i+1} ---")
    print(f"Prompt: '{prompt}' ({num_detections} detections)")
    
    # Extract detection data
    dets = annotation_set['detections']
    boxes_stored = torch.tensor([d['box_xywh'] for d in dets])
    logits_stored = torch.tensor([d['confidence'] for d in dets])
    phrases_stored = [d['phrase'] for d in dets]
    
    # Visualize
    visualize_detections(
        image_source, 
        boxes_stored, 
        logits_stored, 
        phrases_stored,
        title=f"Stored Detections for {sample_image_id}\nPrompt: '{prompt}'"
    )

### Step 4: Demonstrate the `inference_and_visualize` Wrapper

# Cell 7: Use the All-in-One Wrapper

print("Demonstrating the `inference_and_visualize` wrapper function...")

# Pick a new image and prompt
another_image_id = experiment_metadata["image_ids"][200]
another_image_path = get_image_id_paths(another_image_id, experiment_metadata)
another_prompt = "dead embryo"

# Run inference and visualize in one step
_ = inference_and_visualize(
    model,
    another_image_path,
    another_prompt,
    box_threshold=0.3,
    text_threshold=0.25
)

In [ ]:
# Cell 8: Simple Summary

print("=== Demo Summary ===")

# Show what we've accomplished
summary = annotations_manager.export_summary()
print(f"✅ Total images with annotations: {summary['total_images']}")
print(f"✅ Total annotation sets: {summary['total_annotations']}")

# Show prompts used for our sample image
prompts_for_sample = annotations_manager.get_all_prompts_for_image(sample_image_id)
print(f"✅ Prompts used for {sample_image_id}: {prompts_for_sample}")

print(f"\n🎉 Demo complete! Annotations saved to: {annotations_path}")
print("\n💡 Key features demonstrated:")
print("  - Load GroundingDINO model")
print("  - Run inference with different prompts")  
print("  - Save annotations to JSON")
print("  - Retrieve and visualize saved annotations")
print("  - Multiple prompts per image support")